# 2. Building predicion model

## 2.1 Data import and preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df_train = pd.read_csv('E:\\PycharmProjects\\Kaggle\\Titanic\\df_train.csv')
df_Kaggle = pd.read_csv('E:\\PycharmProjects\\Kaggle\\Titanic\\df_test.csv')

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Age,Family_size,Ticket_frequency,Survival_rate,Fare_per_person,Sex_2,Embarked_2,Embarked_3,Deck_2,Deck_3,Deck_4,Deck_5,Deck_6,Deck_7,Deck_8,Deck_9
0,1,0.0,3,22.0,2,1,0.0,7.25000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,1.0,1,38.0,2,2,0.5,35.64165,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1.0,3,26.0,1,1,1.0,7.92500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,1.0,1,35.0,2,2,0.5,26.55000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,3,35.0,1,1,0.0,8.05000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
df_Kaggle.head()

,PassengerId,Survived,Pclass,Age,Family_size,Ticket_frequency,Survival_rate,Fare_per_person,Sex_2,Embarked_2,Embarked_3,Deck_2,Deck_3,Deck_4,Deck_5,Deck_6,Deck_7,Deck_8,Deck_9
0,892,NaN,3,34.5,1,1,0.0,7.82920,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,893,NaN,3,47.0,2,1,0.0,7.00000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,894,NaN,2,62.0,1,1,0.0,9.68750,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,895,NaN,3,27.0,1,1,0.0,8.66250,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,896,NaN,3,22.0,3,2,0.5,6.14375,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
drop_columns=['PassengerId', 'Survived']
X = df_train.drop(columns=drop_columns)
y = df_train['Survived']

I will split train data into train and test set in order to check quality of prediction on test set

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_Kaggle = sc.transform(df_Kaggle.drop(columns=drop_columns))

print('X_train shape: {}'.format(X_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(X_test.shape))
print('t_test shape: {}'.format(y_test.shape))

X_train shape: (712, 17)
y_train shape: (712,)
X_test shape: (179, 17)
t_test shape: (179,)


In [8]:
from sklearn.metrics import classification_report, confusion_matrix
def report(y_pred):
    print(classification_report(y_test, y_pred))
    print('\n')
    print(confusion_matrix(y_test, y_pred))

In [9]:
def save(y_pred, name):
    submission = pd.DataFrame({
            "PassengerId": df_Kaggle["PassengerId"],
            "Survived": y_pred
            },dtype=np.int32)
    submission.to_csv(name, index=False)
    return submission

## 2.2. Logistic regression

Since our goal is to predict 1 of 2 output results logistic regression model is my first choice.

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)
y_logreg_Kaggle = logmodel.predict(X_Kaggle)

y_pred_logreg = logmodel.predict(X_test)

In [12]:
report(y_pred_logreg)

              precision    recall  f1-score   support

         0.0       0.98      0.95      0.97       106
         1.0       0.93      0.97      0.95        73

    accuracy                           0.96       179
   macro avg       0.96      0.96      0.96       179
weighted avg       0.96      0.96      0.96       179



[[101   5]
 [  2  71]]


In [13]:
submission = save(y_logreg_Kaggle,'submission_logreg.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


## 2.3 Random forest

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)
y_rfc_Kaggle = rfc.predict(X_Kaggle)

In [16]:
report(y_pred_rfc)

              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96       106
         1.0       0.96      0.93      0.94        73

    accuracy                           0.96       179
   macro avg       0.96      0.95      0.95       179
weighted avg       0.96      0.96      0.96       179



[[103   3]
 [  5  68]]


In [17]:
submission = save(y_rfc_Kaggle,'submission_rfc.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


## 2.4 Decision tree

In [18]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

y_pred_dtree = dtree.predict(X_test)
y_dtree_Kaggle = dtree.predict(X_Kaggle)

In [21]:
report(y_pred_dtree)

              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       106
         1.0       0.99      0.92      0.95        73

    accuracy                           0.96       179
   macro avg       0.97      0.95      0.96       179
weighted avg       0.96      0.96      0.96       179



[[105   1]
 [  6  67]]


In [20]:
submission = save(y_rfc_Kaggle,'submission_dtree.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
